In [1]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import pandas as pd

# Dummy trading context
entry_price = 29500
total_position_size = 100
side = 'long'
leverage = 10

def estimate_pnl(exit_price, size_pct):
    size = total_position_size * (size_pct / 100)
    raw_pnl = (exit_price - entry_price) * (size / entry_price) * leverage
    return round(raw_pnl if side == 'long' else -raw_pnl, 2)

# === Top Left: Control Panel ===
exchange_dropdown = widgets.Dropdown(options=['Binance', 'Bybit'], value='Binance', description='Exchange:')
connect_button = widgets.Button(description='Connect', button_style='success')
connect_output = widgets.Output()
def on_connect(b):
    with connect_output:
        clear_output()
        print(f"Connected to {exchange_dropdown.value} (dummy)")
connect_button.on_click(on_connect)
top_left = widgets.VBox([exchange_dropdown, connect_button, connect_output], layout=widgets.Layout(min_height='100px'))

# === Top Middle: Position Size ===
balance_slider = widgets.FloatSlider(value=50, min=1, max=100, step=1, description='Size (%):')
position_display = widgets.Label(value=f'Est. Size: {total_position_size} USDT (dummy)')
top_middle = widgets.VBox([balance_slider, position_display], layout=widgets.Layout(min_height='100px'))

# === Top Right: Account Info ===
balance_label = widgets.Label(value="Balance: 1000 USDT (dummy)")
leverage_label = widgets.Label(value="Leverage: 10x (dummy)")
refresh_button = widgets.Button(description='🔄 Refresh')
def refresh_account(b):
    balance_label.value = "Balance: 1000 USDT (refreshed)"
    leverage_label.value = "Leverage: 10x (refreshed)"
refresh_button.on_click(refresh_account)
top_right = widgets.VBox([balance_label, leverage_label, refresh_button], layout=widgets.Layout(min_height='100px'))

# === Bottom Left: Entry Ladder ===
entry_start = widgets.FloatText(value=30000, description="Start Price:", layout=widgets.Layout(width='160px'))
entry_end = widgets.FloatText(value=29000, description="End Price:", layout=widgets.Layout(width='160px'))
entry_steps = widgets.BoundedIntText(value=5, min=1, max=50, step=1, description='Steps:', layout=widgets.Layout(width='120px'))
entry_dist = widgets.Dropdown(options=['uniform', 'linear+', 'linear−'], description='', layout=widgets.Layout(width='80px'))
entry_preview_button = widgets.Button(description="Preview Ladder", layout=widgets.Layout(width='160px'))
entry_output = widgets.Output()

def preview_entry_ladder(b):
    with entry_output:
        clear_output()
        prices = [entry_start.value - i * ((entry_start.value - entry_end.value) / max(1, entry_steps.value - 1)) for i in range(entry_steps.value)]
        sizes = [round(total_position_size / entry_steps.value, 2)] * entry_steps.value
        df = pd.DataFrame({'Price': prices, 'Size': sizes})
        display(df)

entry_preview_button.on_click(preview_entry_ladder)
entry_panel = widgets.VBox([
    entry_start, entry_end,
    widgets.HBox([entry_steps, entry_dist]),
    entry_preview_button,
    entry_output
], layout=widgets.Layout(min_height='260px'))

# === Bottom Middle: Stop Loss Ladder ===
sl_start = widgets.FloatText(value=28000, description="Start SL:", layout=widgets.Layout(width='160px'))
sl_end = widgets.FloatText(value=27000, description="End SL:", layout=widgets.Layout(width='160px'))
sl_steps = widgets.BoundedIntText(value=2, min=1, max=20, step=1, description='Steps:', layout=widgets.Layout(width='120px'))
sl_dist = widgets.Dropdown(options=['uniform', 'linear+', 'linear−'], description='', layout=widgets.Layout(width='80px'))
sl_preview_button = widgets.Button(description="Preview SL Ladder", layout=widgets.Layout(width='160px'))
sl_output = widgets.Output()

def preview_sl_ladder(b):
    with sl_output:
        clear_output()
        prices = [sl_start.value - i * ((sl_start.value - sl_end.value) / max(1, sl_steps.value - 1)) for i in range(sl_steps.value)]
        size_pct = round(100 / sl_steps.value, 2)
        df = pd.DataFrame({
            'SL Price': prices,
            'Size %': [size_pct] * sl_steps.value,
            'PnL': [estimate_pnl(p, size_pct) for p in prices]
        })
        display(df)

sl_preview_button.on_click(preview_sl_ladder)
sl_panel = widgets.VBox([
    sl_start, sl_end,
    widgets.HBox([sl_steps, sl_dist]),
    sl_preview_button,
    sl_output
], layout=widgets.Layout(min_height='260px'))

# === Bottom Right: TP Panel (Stacked Layout Per Row) ===
tp_rows = []
tp_output = widgets.Output()
tp_box = widgets.VBox(layout=widgets.Layout(align_items='flex-start'))  # Left-align all rows

def add_tp_row(_=None):
    price_input = widgets.FloatText(description='TP:', layout=widgets.Layout(width='160px'))
    size_input = widgets.FloatText(description='%:', layout=widgets.Layout(width='70px'))
    pnl_label = widgets.Label(value='PnL: —', layout=widgets.Layout(width='160px'))

    def update_pnl(change=None):
        try:
            pnl = estimate_pnl(price_input.value, size_input.value)
            pnl_label.value = f'PnL: {pnl} USDT'
        except:
            pnl_label.value = 'PnL: —'

    price_input.observe(update_pnl, names='value')
    size_input.observe(update_pnl, names='value')

    row = widgets.VBox([
        price_input,
        widgets.HBox([size_input, pnl_label])
    ], layout=widgets.Layout(margin='0 0 0 0'))

    tp_rows.append(row)
    tp_box.children = list(tp_rows)

add_tp_button = widgets.Button(description="➕ Add TP Target", layout=widgets.Layout(width='160px'))
add_tp_button.on_click(add_tp_row)

generate_tp_button = widgets.Button(description="Generate TP Plan", layout=widgets.Layout(width='160px'))
def generate_tp_plan(b):
    with tp_output:
        clear_output()
        total_pnl = 0
        print("📝 TP Plan:")
        for row in tp_rows:
            price = row.children[0].value
            size_pct = row.children[1].children[0].value
            pnl = estimate_pnl(price, size_pct)
            total_pnl += pnl
            print(f"- TP at ${price} for {size_pct}% → PnL: {pnl} USDT")
        print(f"\n📊 Total Projected TP PnL: {round(total_pnl, 2)} USDT")

tp_panel = widgets.VBox([
    add_tp_button,
    tp_box,
    generate_tp_button,
    tp_output
], layout=widgets.Layout(min_height='260px', width='360px'))

# === Layout Rows ===
top_row = widgets.HBox([top_left, top_middle, top_right],
                       layout=widgets.Layout(justify_content='space-between', width='100%', margin='10px 0px'))
bottom_row = widgets.HBox([entry_panel, sl_panel, tp_panel],
                          layout=widgets.Layout(justify_content='space-between', width='100%'))

# === Display All ===
display(top_row, bottom_row)
